In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import minkowski
# data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/fifa-20-complete-player-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/fifa-20-complete-player-dataset/players_18.csv
/kaggle/input/fifa-20-complete-player-dataset/players_20.csv
/kaggle/input/fifa-20-complete-player-dataset/players_16.csv
/kaggle/input/fifa-20-complete-player-dataset/players_17.csv
/kaggle/input/fifa-20-complete-player-dataset/players_19.csv
/kaggle/input/fifa-20-complete-player-dataset/players_15.csv
/kaggle/input/fifa-20-complete-player-dataset/teams_and_leagues.csv


In [4]:
#Cleaning of data

def transform_data(data):
    data = data.drop(["player_url","dob","real_face","player_traits","long_name","player_tags","body_type"],axis=1)
    data.fillna(value=0, axis=0, inplace=True)

    data["player_positions"] = data["player_positions"].apply(lambda x:x.lower())
    data["player_positions"] = data["player_positions"].apply(lambda x: x.split(","))
    data['gk'] = data['overall'] * data['player_positions'].apply(lambda x: 1 if 'GK' in x else 0)
    
    non_numeric = ["short_name","nationality","club","preferred_foot","work_rate","team_position","loaned_from","joined","nation_position","player_positions"]
    
    for i in data:
        if data[i].dtype == "object" and i not in non_numeric:
            data[i] = data[i].apply(lambda x: eval(str(x)))

    return(data)



In [5]:
#Reading all years values
data_20 = pd.read_csv("../input/fifa-20-complete-player-dataset/players_20.csv")
data_19 = pd.read_csv("../input/fifa-20-complete-player-dataset/players_19.csv")
data_18 = pd.read_csv("../input/fifa-20-complete-player-dataset/players_18.csv")
data_17 = pd.read_csv("../input/fifa-20-complete-player-dataset/players_17.csv")
data_16 = pd.read_csv("../input/fifa-20-complete-player-dataset/players_16.csv")

#Cleaning all the values
data_20 = transform_data(data_20)
data_19 = transform_data(data_19)
data_18 = transform_data(data_18)
data_17 = transform_data(data_17)
data_16 = transform_data(data_16)

#Storing the sofifa_id
data_16_s = list(data_16["sofifa_id"])
data_17_s = list(data_17["sofifa_id"])
data_18_s = list(data_18["sofifa_id"])
data_19_s = list(data_19["sofifa_id"])


#Attributes which are non-numeric and will not be considered for average
non_numeric = ["short_name","nationality","club","preferred_foot","work_rate","team_position","loaned_from","joined","nation_position","player_positions"]
cant_consider = ["value_eur","sofifa_id","height_cm","age","weight_kg","potential","wage_eur","international_reputation","weak_foot","skill_moves","release_clause_eur","team_jersey_number","nation_jersey_number","contract_valid_until"]

data = data_20
print(data.head())
for i in data:
    if i not in non_numeric and i not in cant_consider:
        print(i,type(data[i][0]))

#Weights
w1 = 5/15
w2 = 4/15
w3 = 3/15
w4 = 2/15
w5 = 1/15



#Computation simple weighted average
for i in data:
    if i not in non_numeric and i not in cant_consider:
        print(i)
        for j in data["sofifa_id"]:
            z = data.index[data["sofifa_id"] == j].tolist()[0]
            x = w1*data.iloc[z][i]
            n = 1
            if j in data_19_s:
                x+= w2*data.iloc[data_19_s.index(j)][i]
            else:
                n-=w2
            if j in data_18_s:
                x+= w3*data.iloc[data_18_s.index(j)][i]
            else:
                n-=w3
            if j in data_17_s:
                x+= w4*data.iloc[data_17_s.index(j)][i]
            else:
                n-=w4
            
            if j in data_16_s:
                x+= w5*data.iloc[data_16_s.index(j)][i]
            else:
                n-=w5
            
            
            data[i][z] = x/n
            
print(data)

    

   sofifa_id         short_name  age  height_cm  weight_kg nationality  \
0     158023           L. Messi   32        170         72   Argentina   
1      20801  Cristiano Ronaldo   34        187         83    Portugal   
2     190871          Neymar Jr   27        175         68      Brazil   
3     200389           J. Oblak   26        188         87    Slovenia   
4     183277          E. Hazard   28        175         74     Belgium   

                  club  overall  potential  value_eur  ...  ldm cdm rdm  rwb  \
0         FC Barcelona       94         94   95500000  ...   68  68  68   70   
1             Juventus       93         93   58500000  ...   64  64  64   68   
2  Paris Saint-Germain       92         92  105500000  ...   64  64  64   69   
3      Atlético Madrid       91         93   77500000  ...    0   0   0    0   
4          Real Madrid       91         91   90000000  ...   66  66  66   69   

   lb  lcb  cb  rcb  rb  gk  
0  65   54  54   54  65   0  
1  64   56  56

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


pace
shooting
passing
dribbling
defending
physic
gk_diving
gk_handling
gk_kicking
gk_reflexes
gk_speed
gk_positioning
attacking_crossing
attacking_finishing
attacking_heading_accuracy
attacking_short_passing
attacking_volleys
skill_dribbling
skill_curve
skill_fk_accuracy
skill_long_passing
skill_ball_control
movement_acceleration
movement_sprint_speed
movement_agility
movement_reactions
movement_balance
power_shot_power
power_jumping
power_stamina
power_strength
power_long_shots
mentality_aggression
mentality_interceptions
mentality_positioning
mentality_vision
mentality_penalties
mentality_composure
defending_marking
defending_standing_tackle
defending_sliding_tackle
goalkeeping_diving
goalkeeping_handling
goalkeeping_kicking
goalkeeping_positioning
goalkeeping_reflexes
ls
st
rs
lw
lf
cf
rf
rw
lam
cam
ram
lm
lcm
cm
rcm
rm
lwb
ldm
cdm
rdm
rwb
lb
lcb
cb
rcb
rb
gk
       sofifa_id         short_name  age  height_cm  weight_kg nationality  \
0         158023           L. Messi   32       

In [ ]:
print(158023 in list(data["sofifa_id"]))

In [6]:
#Selecting the playing 11
formations = dict()
formations = {
        '433': ['gk', 'lb', 'cb', 'cb', 'rb', 'cm', 'cm', 'cam', 'lw', 'rw', 'st'],
        '442': ['gk', 'lb', 'cb', 'cb', 'rb', 'cdm', 'rm', 'lm', 'cam', 'st', 'st'],
        '352': ['gk', 'cb', 'cb', 'cm', 'cm', 'cm', 'rwb', 'lwb', 'cam', 'cf', 'st']
    }

#Positioned according to the similarity (HARD-CODED)
center = ['cb', 'cdm', 'cm', 'cam', 'cf', 'st']
side =  ['lb', 'rb', 'lwb', 'rwb', 'lm', 'rm', 'lw', 'rw']

current_formation = "433"
current_club = "FC Barcelona"

YOUNG_PLAYER_MARGIN = 25
PLAYER_AGE_PENALTY = 0.05

#Team attributes common to each player
attributes = ["age","pace","shooting","passing","dribbling","defending"]
#print(formations[current_formation])

team = data.loc[data["club"] == current_club]



def similar_positions(pos):
    sim_pos = []
    s = 0
    if pos in center:
        s = center.index(pos) #Gets the corresponding index
        if s - 1 > 0:
            sim_pos.append(center[s-1])
        if s + 1 < len(center):
            sim_pos.append(center[s+1])
    elif pos in side:
        s = side.index(pos)
        if s - 2 > 0:
            sim_pos.append(side[s-2])
        if s + 2 < len(side):
            sim_pos.append(side[s+2])
    return(sim_pos)
    
#if the position is present put it into the first_11 else find similar positions and put that player into the team
def starting_11(formation,team):
    form = formations[formation]      
    first_11 = pd.DataFrame()
    for i in form:
        l = team.loc[team["player_positions"].apply(lambda x:i in x)]
        if len(l) == 0:
            new_pos = similar_positions(i)
            #print(new_pos)
            for j in new_pos:
                l.append(team.loc[team["player_positions"].apply(lambda x:j in x)])
        if len(l) == 0:
            l = team

        player = l[i].idxmax()
        player_pd = l.loc[[player]]
        player_pd["Position"] = i
        first_11 = first_11.append(player_pd)
        team.drop([player],axis=0,inplace = True)
    
    return(first_11)


        
players = starting_11(current_formation,team)
print(players)

     sofifa_id     short_name  age  height_cm  weight_kg nationality  \
6       192448  M. ter Stegen   27        187         85     Germany   
602     241184   Junior Firpo   22        184         78       Spain   
59      205600      S. Umtiti   25        182         75      France   
84      220440     C. Lenglet   24        186         81      France   
249     227928  Nélson Semedo   25        177         67    Portugal   
115     230658         Arthur   22        171         73      Brazil   
64      168651     I. Rakitić   31        184         78     Croatia   
344     201400        Rafinha   26        174         71      Brazil   
0       158023       L. Messi   32        170         72   Argentina   
110     231443     O. Dembélé   22        178         67      France   
19      176580      L. Suárez   32        182         86     Uruguay   

             club  overall  potential  value_eur  ...  cdm rdm rwb  lb  lcb  \
6    FC Barcelona       87         93   67500000  ...   

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
#Finding the worst players by sorting the players

def sort_first_eleven(players):
    players['true_overall'] = np.where(players['age'] < YOUNG_PLAYER_MARGIN,
                                       (players['potential'] + players['overall'])/2,
                                       players['overall'])
    players['diff'] = players['true_overall'] - players['true_overall'].mean()
    avg_wage = players['wage_eur'].mean()

    players['wage_diff'] = (players['wage_eur'] - avg_wage) / (players['wage_eur'].max() - players['wage_eur'].min())
    players['diff'] = np.where((players['diff'] > 0) & (players['age'] > YOUNG_PLAYER_MARGIN),
                               players['diff'] * (1 - (players['age'] - YOUNG_PLAYER_MARGIN) * PLAYER_AGE_PENALTY),
                               players['diff'])

    players['diff'] = np.where((players['diff'] < 0) & (players['age'] > YOUNG_PLAYER_MARGIN),
                               players['diff'] * (0.5 + (players['age'] - YOUNG_PLAYER_MARGIN) * PLAYER_AGE_PENALTY),
                               players['diff'])


    players['diff'] = (players['diff'] - players['diff'].mean()) / (players['diff'].max() - players['diff'].min())
    # print(players.sort_values('diff')[['diff', 'Name', 'wage_diff']])
    players['diff'] = 0.7*players['diff'] + 0.3*(-1 * players['wage_diff'])
    # print(players.sort_values('diff')[['diff', 'Name', 'wage_diff']])
    return players.sort_values('diff')

playing_11 = sort_first_eleven(players)

In [10]:
#Creating the joint profile of the team
def joint_profile(players):
    print()
    players_filtered = players[players["gk"] ==0]
    avg_stats = players_filtered[attributes].mean()
    std_stats = players_filtered[attributes].std()
    return(avg_stats,std_stats)



#Filtering the players based on some constriaints
def filtering(player,pos,data):
    
    filtered_data = data.loc[data["player_positions"].apply(lambda x: pos in x)]
    filtered_data = filtered_data[filtered_data["club"] != player["club"]]
    filtered_data = filtered_data[(filtered_data[pos] > player[pos]) | ((filtered_data["age"] < YOUNG_PLAYER_MARGIN) & (filtered_data["potential"] > player["potential"]+3))]
    
    filtered_data = filtered_data[filtered_data["wage_eur"] < 2*player["wage_eur"]]
    
    return filtered_data
    

In [21]:
#Differnent distances

def find_similarity(hyp_player,filtered_list,similarity="pearson"):
    avg = hyp_player[0]
    sd = hyp_player[1]
    avg["age"]  = 21
    
    sd = 1/sd
    sd = sd / sd.sum()
    
    avg = avg.values.reshape(-1,len(avg))
    f_list = filtered_list[attributes]
    f_list = f_list.mul(sd,axis = 1)
    avg_s = pd.Series(avg.flatten(),index=f_list.columns)
    
    if similarity == "pearson":
        pearson_sim = f_list.corrwith(avg_s,axis=1)
        filtered_list['pearson'] = pearson_sim
        return filtered_list.sort_values(['pearson'], ascending=False)
    
    if similarity == "cosine":
        cos_sim = cosine_similarity(f_list,avg)
        filtered_list['cosine'] = pd.Series([x for row in cos_sim for x in row], index=filtered_list.index)
        return filtered_list.sort_values(['cosine'], ascending=False)
    
    if similarity == 'minkowski':
        minkowski_sim = []
        for index, row in f_list.iterrows():
            minkowski_sim.append(minkowski(row.values, avg.flatten(), p=2))

        filtered_list['minkowski'] = pd.Series(minkowski_sim, index=filtered_list.index)
        return filtered_list.sort_values(['minkowski'], ascending=False)
        
        
    

In [23]:
#Model testing

current_formation = "433"
current_club = "Real Madrid"

team = data.loc[data["club"] == current_club]

first_11 = starting_11(current_formation,team)
first_11 = sort_first_eleven(first_11)
hyp_player = joint_profile(first_11)

worst_player = first_11.iloc[0]

print(worst_player)

filtered_list = filtering(worst_player,worst_player["Position"],data)

per = find_similarity(hyp_player,filtered_list,similarity="pearson")
c = find_similarity(hyp_player,filtered_list,similarity="cosine")
m = find_similarity(hyp_player,filtered_list,similarity="minkowski")
print(per[0:6])
print(c[0:6])
print(m[0:6])


/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,



sofifa_id              227928
short_name      Nélson Semedo
age                        25
height_cm                 177
weight_kg                  67
                    ...      
gk                          0
Position                   rb
true_overall               80
diff                -0.275264
wage_diff           -0.188811
Name: 249, Length: 102, dtype: object
      sofifa_id           short_name  age  height_cm  weight_kg nationality  \
160      231281  T. Alexander-Arnold   20        180         69     England   
1024     236401          N. Mazraoui   21        183         62     Morocco   
4059     247204              Emerson   20        181         79      Brazil   
1708     230147             F. Sacko   24        179         64        Mali   
94       204963             Carvajal   27        173         73       Spain   
112      210514         João Cancelo   25        182         74    Portugal   

                   club  overall  potential  value_eur  ...  cdm rdm rwb  lb 